In [0]:
from pyspark.sql import functions as F
from functools import reduce

filepath = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'

Nous chargeons le fichier dans un dataframe pyspark et nous affichons ses 5 premiers éléments pour se faire une idée du contenu.

Dans le cas présent il n'y a que 2 colonnes : une colonne "id" et une colonne "data". Cette dernière semble contenir l'intégralité des données exploitables.

In [0]:
df = (spark.read.format('json')\
             .option('header', 'true')\
             .load(filepath))

df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



Nous nous intéressons maintenant au contenu de la colonne "data", en commençant par afficher le schéma du dataframe.

Comme pressenti, la colonne "data" contient les données qui nous intéressent. Elle contient aussi un champ "tags" qui lui-même renferme plus de 400 sous-colonnes.

Après renseignement, les champs "tags" correspondent aux avis des joueurs (sous la forme de votes) concernant les catégories auxquelles un jeu appartient. Ces champs sont à contraster avec ceux apparaissant dans la colonne "genre" qui contient la même information mais du point de vue de l'éditeur du jeu. Source : https://store.steampowered.com/tag/

In [0]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

L'objectif est désormais d'aplatir le contenu de la colonne "data" pour le répartir en plusieurs colonnes (pour rendre ces informations exploitables).

La première étape est de générer automatiquement une liste du nom des colonnes à récupérer.

In [0]:
field_names = [field.name for field in next(field for field in df.schema.fields if field.name=="data").dataType.fields]

for field in field_names:
    print(field)

appid
categories
ccu
developer
discount
genre
header_image
initialprice
languages
name
negative
owners
platforms
positive
price
publisher
release_date
required_age
short_description
tags
type
website


Nous pouvons donc aplatir la colonne "data" en fonction des noms de champs récupérés précédemment, puis nous débarasser des colonnes "data" (dont nous venons d'extraire les données) et "tags" (que nous n'utiliserons pas pour le moment). La fonction reduce() est particulièrement bien adaptée, nous allons donc l'utiliser.

In [0]:
def flatten(memo_df, my_col):
    return memo_df.withColumn(my_col, F.col("data").getField(my_col))

flat_df = reduce(flatten, field_names, df).drop('data').drop('tags')

In [0]:
flat_df.printSchema()
display(flat_df.limit(10))

root
 |-- id: string (nullable = true)
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- type: string (nul

id,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,type,website
10,10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000","List(true, true, true)",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,game,
1000000,1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/1000000/header.jpg?t=1655723048,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","List(false, false, true)",27,999,PsychoFlux Entertainment,2021/05/14,0,"ASCENXION is a 2D shoot 'em up game where you explore the field to progress. Players must overcome puzzles, traps, elite units, boss fights, and other various obstacles while navigating the field. Grow stronger through rewards earned, to uncover the truth of this world.",game,
1000010,1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/1000010/header.jpg?t=1655724189,1999,"Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil",Crown Trick,646,"200,000 .. 500,000","List(false, false, true)",4032,599,"Team17, NEXT Studios",2020/10/16,0,"Enter a labyrinth that moves as you move, where mastering the elements is key to defeating enemies and uncovering the mysteries of this underground world. With a new experience awaiting every time you enter the dungeon, let the power bestowed by the crown guide you in this challenging adventure!",game,
1000030,1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/1000030/header.jpg?t=1660866300,1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000","List(false, true, true)",1575,1999,Vertigo Gaming Inc.,2020/10/14,0,"Cook, serve and manage your food truck as you dish out hundreds of different foods across war-torn America in this massive sequel to the million-selling series!",game,http://www.cookservedelicious.com
1000040,1000040,List(Single-player),0,DoubleC Games,0,"Action, Casual, Indie, Simulation",https://cdn.akamai.steamstatic.com/steam/apps/1000040/header.jpg?t=1627033870,199,Simplified Chinese,细胞战争,1,"0 .. 20,000","List(false, false, true)",0,199,DoubleC Games,2019/03/30,0,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通过与细胞之间的战斗来获得基因变异点数和进入下一关的资格，每种细菌病毒都有独特的能力和攻击效果，你是否可以破坏五大器官并占领人体呢！？,game,
1000080,1000080,"List(Multi-player, Single-player, Steam Achievements, Full controller support, Steam Trading Cards)",3,IndieLeague Studio,60,"Action, Adventure, Indie, RPG",https://cdn.akamai.steamstatic.com/steam/apps/1000080/header.jpg?t=1667062553,1999,"Simplified Chinese, English, Traditional Chinese, Japanese, Korean",Zengeon,462,"100,000 .. 200,000","List(false, true, true)",1018,799,2P Games,2019/06/24,0,Zengeon is an anime infused Action RPG and Roguelite with a selection of unique ch

In [0]:
display(flat_df.summary())

summary,id,appid,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age,short_description,type,website
count,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691
mean,1025603.0926720655,1025603.0926720655,138.9596164550825,67392.0,2.603777989262179,null,null,797.5663033524268,null,Infinity,241.8376937027527,null,1470.8755992889335,773.2849832109317,2001.0,null,0.1978882344490734,null,null,null
stddev,522784.96832834504,522784.96832834504,6002.067909130765,210681.70504552333,12.887080174743176,null,null,1104.762477841338,null,NaN,5765.413761559615,null,30982.733479534887,1093.13458272345,1921.8937275510318,null,2.2962924614818236,null,null,null
min,10,10,0,,0,,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,0,,Fieldrunners 2,0,"0 .. 20,000",0,0,,,0,,game,
25%,598530.0,598530,0,125.0,0.0,null,null,199.0,null,103.0,1,null,5,129.0,123.0,null,0.0,null,null,null
50%,986320.0,986320,0,773.0,0.0,null,null,499.0,null,1010.0,6,null,19,499.0,2054.0,null,0.0,null,null,null
75%,1427920.0,1427920,1,2015.0,0.0,null,null,999.0,null,2021.0,32,null,106,999.0,3909.0,null,0.0,null,null,null
max,999990,2190950,874053,＼上／,90,Web Publishing,https://cdn.akamai.steamstatic.com/steam/apps/999990/header.jpg?t=1610733322,9999,Turkish,～Daydream～蝶が舞う頃に,908515,"500,000 .. 1,000,000",5943345,9999,Ｌｅｍｏｎ Ｂａｌｍ,2022/11/7,MA 15+,"🚗 Take part in a roller coaster of emotions with Louise embarking on a road trip of a lifetime through the late 1960s USA, trying to show her son Mitch how to navigate the often cruel modern world. Your choices matter! ✅",hardware,www.windybeard.com


Nous pouvons désormais nous intéresser à chacune des colonnes et décider si elle sera à priori utile ou non pour la suite.

In [0]:
# l'information contenue dans id et appid est-elle redondante ? OUI

print(flat_df.select('id').distinct().count())
print(flat_df.select('appid').distinct().count())

55691
55691


In [0]:
# tous les jeux ont-ils une image de présentation ? OUI

flat_df.filter(F.col('header_image').isNull()).count()

Out[9]: 0

In [0]:
# que contient vraiment la colonne 'owmers' ?
# nous nous apercevons que cette colonne comporte 13 classes signalant le nombre d'unités du jeu vendues
# par contre si 'owners' vaut "100,000 .. 200,000", entre 100 000 et 200 000 exemplaires du jeu ont été vendus

display(flat_df.select('owners').distinct())

owners
"100,000 .. 200,000"
"1,000,000 .. 2,000,000"
"20,000,000 .. 50,000,000"
"5,000,000 .. 10,000,000"
"0 .. 20,000"
"20,000 .. 50,000"
"2,000,000 .. 5,000,000"
"50,000,000 .. 100,000,000"
"200,000,000 .. 500,000,000"
"500,000 .. 1,000,000"


In [0]:
# dans quel format est la date de sortie ? STRING

flat_df.select('release_date').describe()

Out[11]: DataFrame[summary: string, release_date: string]

In [0]:
# que contient le champ required_age ? une information a priori peu pertinente, mais à approfondir

display(flat_df.groupBy('required_age').count())
display(flat_df.groupBy('required_age').max('ccu'))

required_age,count
7,2
15,264
3,3
8,3
16,38
35,1
0,55030
21+,1
5,1
18,223


required_age,max(ccu)
7,34
15,273088
3,0
8,365
16,2429
35,2
0,874053
21+,0
5,0
18,50989


In [0]:
# tous les jeux ont-ils un type et si oui lequel ?
# tous les jeux sont des jeux (!!!) sauf un (Steamlink)

display(flat_df.groupBy('type').count())
display(flat_df.filter("type == 'hardware'"))

type,count
hardware,1
game,55690


id,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,type,website
353380,353380,"List(Full controller support, Remote Play Together)",0,,0,,https://cdn.akamai.steamstatic.com/steam/apps/353380/header.jpg?t=1617990330,0,,Steam Link,1771,"500,000 .. 1,000,000","List(true, true, true)",5803,0,Anima Locus,2015/11/10,0,"Extend your Steam gaming experience to your mobile device, TV, or another PC - all you need is a local network or internet connection. In addition, the Steam Link app now supports Remote Play Together. Now you can join games hosted on a friend’s PC just by clicking a link.",hardware,https://store.steampowered.com/remoteplay



### id / appid

Il y a autant d'id distinctes que d'appid distinctes que de lignes dans notre table. Cette information est donc redondante et semble un id interne dans Steam. Cela ne nous sera a priori pas nécessaire, nous pouvons donc retirer cette colonne et garder seulement id.

Plutôt que de retirer les colonnes une par une tout au long de cette analyse, nous allons ajouter son nom à une liste que nous utiliserons avec la commande .drop() pour nettoyer la table à la fin.

### categories

Une liste de catégories concernant le type de jeu, par exemple PvP ou multiplayer. Cette information est très importante pour notre étude mais nous ne pouvons pas y accéder facilement. Il faudra la distribuer via une duplication des lignes, potentiellement dans un deuxième temps.

### ccu

CCU signifie "Concurrent Connected Users", dans notre datatrame c'est le nombre maximal de joueurs en train de jouer au jeu au même moment. C'est un indicateur important du succès d'un jeu à son apogée, nous décidons donc de conserver cette colonne.

### developer / name / publisher

Le nom du jeu ainsi que celui du studio ayant développé le jeu et celui de l'éditeur, des informations très importantes que nous conservons donc.

### discount

Indique en % si une réduction est appliquée à l'heure actuelle. Information importante pour les politiques de prix.

### genre

Une string contenant une liste des catégories auxquelles ce jeu appartient (RPG, FPS, Horreur...), une information importante pour pouvoir segmenter l'information. Nous devrons distribuer cette liste dans un second temps ("explode").

### header_image

Une image associée au jeu, tous les jeux en ont une. Cette information ne semble pas pertinente.

### initalprice / price

Informations utiles pour cette étude.

### languages

Liste des langues dans lequel le jeu est disponible. A conserver pour le moment, à aplatir quand nous en aurons besoin.

### negative / positive

Nombres de votes négatifs ou positifs qu'un jeu a reçu de la part des joueurs. Cette information est très utile pour déterminer le succès d'un jeu et devrait nous permettre d'attribuer une note au jeu.

### owners

Nombre d'exemplaires du jeu vendus, divisés en 13 classes. Il pourrait être utile de diviser ces classes en 2 colonnes min/max pour faire des calculs (par exemple estimation du chiffre d'affaires)

### platforms

Une liste de 3 booléens indiquant la disponibilité du jeu sur les 3 plateformes principales (ordre de la liste : linux, mac, windows). À répartir sur 3 colonnes distinctes. 

### release_date

Date de sortie du jeu, à transformer en format date correct à partir d'un format string.

### required_age

Le champ required_age est composé de plusieurs catégories, mais une d'entre elle ('0') domine les débats (environ 55 030 jeux sur les 55 691 jeux présents dans notre dataframe). Néanmoins des jeux ayant des restrictions d'âge atteignent des ccu importants (de l'ordre de 50 000 à 100 000 ccu, alors que le record est de 875 000 environ), il pourrait être intéresser de regarder quels jeux sont concernés.

### short_description

Une description du jeu qui ne présente que peu d'intérêt pour nous a priori (et occupe comparativement beaucoup de place).

### type

Tous les jeux référencés sont du type "game" sauf un, nous allons donc ignorer cette colonne

### website

Le site web du jeu, a priori sans intérêt pour nous.

Nous procédons au retrait des données sans intérêt ainsi qu'à la tranformation d'autres colonnes :
- passage de la colonne 'release_date' au format date
- création de 3 colonnes à partir de la colonne 'platforms'

In [0]:
to_be_dropped = ['appid', 'header_image', 'short_description', 'type', 'website']

flat_df = flat_df.drop(*to_be_dropped)

In [0]:
flat_df = flat_df.withColumn("release_date", F.to_date(F.col("release_date"), format="y/M/d"))

In [0]:
platforms = ['linux', 'mac', 'windows']

for platform in platforms:
    flat_df = flat_df.withColumn(platform, F.col("platforms").getField(platform))

flat_df = flat_df.drop('platforms')

In [0]:
flat_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- required_age: string (nullable = true)
 |-- linux: boolean (nullable = true)
 |-- mac: boolean (nullable = true)
 |-- windows: boolean (nullable = true)




## Analyse des données générales

Voici quelques questions que nous sommes susceptibles de nous poser :

### Question 1 : Quel éditeur a publié le plus de jeux sur Steam ?

Cette question peut aussi se poser pour les studios de développement.

### Question 2 : Quels sont les jeux les mieux notés ?

Steam ne publie pas de note à proprement parler, nous devrons donc la déduire des autres colonnes. Nous retenons la formule suivante : note = 100 * positive / (positive + negative)
La note est donc le pourcentage de votes positifs. Cependant cette note peut facilement mettre en avant des jeux ayant très peu de notes. Pour sélectionner les jeux les mieux notés il faudrait donc veiller à créer un sous-ensemble de jeux "qualifiés", par exemple les jeux ayant reçu un nombre minimum de votes au total.

Par ailleurs un jue bien noté n'est pas forcément un jeu populaire. Nous nous intéresserons donc aussi au 'ccu' (concurrent connected users) qui est potentiellement plus révélateur du succès réel d'un jeu.

Pour finir, le nombre d'exemplaires vendus est aussi un bon indicateur et nous regarderons si l'on trouve les mêmes jeux.

### Question 3 : Y a-t-il des années pendant lesquelles plus de jeux sont sortis ? Ou d'autres périodes, comme celle du Covid ?

Pour répondre à cette question nous commencerons par regarder la répartition par années puis lors d'événements spécifiques (Covid) ou à l'intérieur d'une année (par exemple la période de Noël).

### Question 4 : Comment les prix des jeux sont-ils distribués et qu'en est-il des réductions de prix ?

Nous regarderons le contenu des colonnes 'price' et 'discount'.

### Question 5 : Quels langages sont les plus représentés ?

Il sera temps de regarder la colonne 'languages' et de récupérer les différentes valeurs présentes dans la liste. Cependant cela ne correspond pas forcément à la langue d'origine du jeu (ou du studio, ou de l'éditeur). Il sera intéressant de regarder si beaucoup de jeux sont disponibles dans de nombreuses langues.

### Question 6 : Y a-t-il beaucoup de jeux interdits aux moins de 18 ans ? au moins de 16 ans ?

Cela sera une opportunité pour regarder de quel type de jeux il s'agit et d'évaluer leur succès.

### Question 1 : Quel éditeur a publié le plus de jeux sur Steam ?

In [0]:
print("Top 10 des éditeurs (par nombre de jeux)")
display(flat_df.groupBy('publisher').count().sort('count', ascending=False).take(10))
print("Top 10 des dévelopeurs (par nombre de jeux)")
display(flat_df.groupBy('developer').count().sort('count', ascending=False).take(10))

Top 10 des éditeurs (par nombre de jeux)


publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132
,132
Ubisoft,127


Top 10 des dévelopeurs (par nombre de jeux)


developer,count
Choice of Games,140
,127
Creobit,122
Laush Dmitriy Sergeevich,108
Sokpop Collective,98
"KOEI TECMO GAMES CO., LTD.",90
Reforged Group,89
Boogygames Studios,80
Hosted Games,79
Elephant Games,75


Nous remarquons que les plus gros éditeurs et les plus gros studios ont publié plusieurs centaines de jeux sur Steam.

L'éditeur "Big Fish Game" a publié largement plus de jeux que n'importe qui d'autre sur Steam (plus de 2 fois plus que l'éditeur arrivant en seconde position) mais il n'apparaît pas dans le top 10 des dévelopeurs.

À l'inverse "Choice of Games" a édité et développé 140 jeux : ce studio a (a priori) édité lui-même tous les jeux qu'il a développés. C'est aussi le studio ayant développé le plus de jeux.

Pour mieux comprendre le lien entre éditeur et dévelopeur, penchons-nous par exemple sur le cas de Square Enix :

In [0]:
display(flat_df.select('publisher', 'developer').filter(F.col('publisher') == 'Square Enix').limit(10))
display(flat_df.select('publisher').filter((F.col('publisher') == 'Square Enix') & (F.col('developer') == 'Square Enix')).count())

publisher,developer
Square Enix,Square Enix
Square Enix,"Square Enix, KOEI TECMO GAMES CO., LTD."
Square Enix,Eidos-Montréal
Square Enix,Original Fire Games
Square Enix,"Square Enix, Toylogic Inc."
Square Enix,Square Enix
Square Enix,"Square Enix, D4Enterprise Co.,Ltd."
Square Enix,Square Enix
Square Enix,Square Enix
Square Enix,Square Enix


42

Sur 141 jeux, Square Enix en a développé 41 en solo mais le nom de Square Enix apparaît souvent conjoitement avec un autre studio du côté développement.

Nous pouvons aussi regarder le nombre de jeux pour lequel l'éditeur est le même que le développeur :

In [0]:
display(flat_df.groupby('publisher', 'developer').count().sort('count', ascending = False).limit(10))

publisher,developer,count
Choice of Games,Choice of Games,140
Laush Studio,Laush Dmitriy Sergeevich,105
Sokpop Collective,Sokpop Collective,98
8floor,Creobit,94
Reforged Group,Reforged Group,89
"KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.",86
Hosted Games,Hosted Games,79
Boogygames Studios,Boogygames Studios,78
Big Fish Games,Elephant Games,73
Blender Games,Blender Games,70


### Question 2 : Quels sont les jeux les mieux notés ?



In [0]:
# Réponse 1 : jeux ayant le plus d'avis positifs
# Réponse 2 : jeux ayant la meilleure "note" : 100 * votes_positifs / (nombre total de votes)
# Réponse 3 : jeux ayant eu le plus de ccu (concurrent connected users)

# Nous commençons par ajouter une colonne "rating" avec notre note
# Nous éliminons tous les jeux n'ayant pas atteint une certaine quantité de votes positifs (outliers dans ce cas)

min_nb_reviews = 50000

top_df = flat_df \
    .select('*')\
    .filter((flat_df.positive + flat_df.negative) > min_nb_reviews)\
    .withColumn('rating', F.format_number(100 * flat_df.positive / (flat_df.positive + flat_df.negative), 2))

# Réponse 1

top_df1 = top_df.select('name', 'positive', 'rating', 'ccu').sort('positive', ascending = False)

# Réponse 2

top_df2 = top_df.select('name', 'positive', 'rating', 'ccu'). sort('rating', ascending = False)

# Réponse 3

top_df3 = top_df.select('name', 'positive', 'rating', 'ccu').sort('ccu', ascending = False)

In [0]:
print("Top 20 des jeux ayant eu le plus de votes positifs")
top_df1.show(20)
print("Top 20 des jeux ayant eu la meilleure note d'après notre définition")
top_df2.show(20)
print("Top 20 des jeux ayant eu le nombre de joueurs au même temps le plus élevé")
top_df3.show(20)

Top 20 des jeux ayant eu le plus de votes positifs
+--------------------+--------+------+------+
|                name|positive|rating|   ccu|
+--------------------+--------+------+------+
|Counter-Strike: G...| 5943345| 88.31|874053|
|              Dota 2| 1534895| 82.84|852995|
|  Grand Theft Auto V| 1229265| 85.21|140671|
| PUBG: BATTLEGROUNDS| 1185361| 56.61|339287|
|            Terraria| 1014711| 97.84| 58984|
|Tom Clancy's Rain...|  942910| 86.81| 31239|
|         Garry's Mod|  861240| 96.63| 39043|
|     Team Fortress 2|  846407| 93.65|108900|
|                Rust|  732513| 86.72|121146|
|       Left 4 Dead 2|  643836| 97.45| 19084|
|The Witcher 3: Wi...|  632627| 96.16| 22812|
|            Among Us|  586302| 91.79|  8479|
|Euro Truck Simula...|  572368| 97.34| 39651|
|    Wallpaper Engine|  561096| 98.07|109428|
|            PAYDAY 2|  532013| 89.40| 40874|
|    Dead by Daylight|  509637| 81.41| 43211|
|      Stardew Valley|  497558| 98.17| 32996|
|       Rocket League|  49649

En conclusion nous pouvons observer qu'il est rare qu'un jeu apparaisse dans les 3 catégories et que le résultat dépendra donc fortement de la métrique retenue.

### Question 3 : Y a-t-il des années pendant lesquelles plus de jeux sont sortis ? Ou d'autres périodes, comme celle du Covid ?

In [0]:
time_df = flat_df \
    .select('*')\
    .withColumn('year', F.year('release_date'))\
    .withColumn('month', F.month('release_date'))

time_df.select('name', 'release_date', 'year', 'month').limit(10)

Out[23]: DataFrame[name: string, release_date: date, year: int, month: int]

In [0]:
display(time_df.groupBy('year').count().sort('year'))

year,count
null,222
1997,2
1998,1
1999,3
2000,2
2001,4
2002,1
2003,3
2004,6
2005,6


Databricks visualization. Run in Databricks to view.

In [0]:
display(time_df.groupBy('month').count().sort('month'))

month,count
null,222
1,3868
2,4214
3,4697
4,4461
5,4719
6,4206
7,4754
8,5064
9,5166


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

À la vue de ces grpahes le mois qui compte le plus de sorties de jeux est le mois d'octobre. Aucun "effet Noël" n'est apparent car les mois de novembre, décembre et janvier semblent plus calmes que la moyenne. Ceci dit il faudrait consulter le département Marketing : il est possible que le mois d'Octobre soit préféré (par rapport à Noël), le temps qu'un jeu accumule suffisamment de bonne réputation, de recommandations ou de joueurs avant Noël.

In [0]:
display(time_df.filter((time_df.month > 2) & (time_df.month < 7) & (time_df.year >= 2019) & (time_df.year <= 2021)).groupBy('month', 'year').count().sort('month', 'year'))

month,year,count
3,2019,559
3,2020,564
3,2021,824
4,2019,603
4,2020,630
4,2021,704
5,2019,645
5,2020,693
5,2021,661
6,2019,483


À la vue de ces chiffres il ne semble pas que le Covid ait eu une quelconque importance, hormis peut-être le mois de mai. Ceci peut potentiellement s'expliquer par le fait que les cycles de développement de jeux vidéo sont très longs et que les dates de sortie sont anticipées des mois voire des années à l'avance. Les éditeurs n'ont donc probablement pas eu le temps de réagir pour sortir plus de jeux, d'autant plus que leurs équipes étaient elles aussi affectées par la situation. On ne note pas non plus de baisse significative pour les mêmes raisons (les jeux étant sortis sur cette période étant prêts depuis longtemps).

Il serait intéressant de regarder l'impact du Covid sur les "concurrent users" mais ces données ne sont pas disponibles dans ce dataset.

### Question 4 : Comment les prix des jeux sont-ils distribués et qu'en est-il des réductions de prix ?

In [0]:
flat_df.select('initialprice').distinct().count()
flat_df.select('price').distinct().count()

Out[27]: 385

In [0]:
display(flat_df.select('initialprice').describe())
display(flat_df.select('price').describe())

summary,initialprice
count,55691
mean,797.5663033524268
stddev,1104.762477841338
min,0
max,9999


summary,price
count,55691
mean,773.2849832109317
stddev,1093.13458272345
min,0
max,9999


In [0]:
display(flat_df.select('initialprice').sort('initialprice', ascending=False).limit(5))

initialprice
9999
9999
9999
9999
9999


Nous pouvons constater que :
- il y a trop de prix différents (203) pour pouvoir faire une analyse pertinente à ce stade
- les prix sont en centimes et non en euros
- les prix vont de 0 à 99.99 euros avec une moyenne de 7.98 et un écart-type de 11.05

Nous allons donc procéder à 2 opérations pour rendre ces données exploitables :
- passer les prix en euros
- les répartir en classes

Ces 2 transformations seront aussi effectuées sur la colonne "price" qui contient le prix actuel (après remise).

In [0]:
#display(flat_df.limit(5))

from pyspark.sql.types import FloatType

price_df = flat_df.withColumn('initialprice', flat_df.initialprice.cast(FloatType()) / 100)\
                  .withColumn('price', flat_df.price / 100)

display(price_df.select('initialprice').sort('initialprice', ascending=False).limit(5))
display(price_df.select('price').sort('price', ascending=False).limit(5))

initialprice
999.0
299.9
269.99
249.0
199.99


price
999.0
299.9
269.99
249.0
199.99


In [0]:
display(price_df.select('initialprice').sort('initialprice', ascending=False).limit(5))

initialprice
999.0
299.9
269.99
249.0
199.99


In [0]:
from pyspark.ml.feature import Bucketizer

my_bucket = Bucketizer()
my_bucket.setSplitsArray([[x for x in range(0, 110, 10)], [x for x in range(0, 110, 10)]])   # classes de prix de 10 en 10 entre 0 et 100 euros
 

price_df = flat_df.withColumn('initialprice', flat_df.initialprice / 100)\
                  .withColumn('price', flat_df.price / 100)

my_bucket.setInputCols(['initialprice', 'price'])
my_bucket.setOutputCols(['init_price_bucket', 'price_bucket'])

price_df2 = my_bucket.transform(price_df)
display(price_df2.select('initialprice', 'price', 'init_price_bucket', 'price_bucket'))

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 89.0 failed 1 times, most recent failure: Lost task 0.0 in stage 89.0 (TID 266) (ip-10-172-249-71.us-west-2.compute.internal executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (Bucketizer$$Lambda$8777/1520516075: (double) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:246)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:761)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1740)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Feature value 199.99 out of Bucketizer bounds [0.0, 100.0]. Check your features, or loosen the lower/upper bound constraints.
	at org.apache.spark.ml.feature.Bucketizer$.binarySearchForBuckets(Bucketizer.scala:296)
	at org.apache.spark.ml.feature.Bucketizer.$anonfun$transform$2(Bucketizer.scala:170)
	at scala.runtime.java8.JFunction1$mcDD$sp.apply(JFunction1$mcDD$sp.java:23)
	... 26 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3470)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:3392)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:3381)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:3381)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1490)
	at org.apache.spark.scheduler.DAGSch

### Question 5 : Quelles langues sont les plus représentées ?

In [0]:
result_lang = flat_df.withColumn('languages_splited', F.explode(F.split(flat_df['languages'], ', '))) \
    .groupBy('languages_splited') \
    .count() \
    .orderBy(F.desc('count'))
    
result_lang.limit(5).toPandas()

,languages_splited,count
0,English,55116
1,German,14019
2,French,13426
3,Russian,12922
4,Simplified Chinese,12782


### Question 6 : Y a-t-il beaucoup de jeux interdits aux moins de 18 ans ? au moins de 16 ans ?

In [0]:
from pyspark.sql.types import LongType

def convert_str_to_int(df,str_col_name):
    df = df.withColumn(str_col_name, df[str_col_name].cast(LongType()))
    return df

def check_type(df,str_col_name):
    print(type(df.select(str_col_name).head()[str_col_name]))

convert_str_to_int(flat_df,'required_age')
check_type(flat_df,'required_age')

<class 'str'>


In [0]:
def distribution(df,select_list_col,groupby_col,order_by_col):
    result_df = df.select(*select_list_col) \
        .groupBy(*groupby_col) \
        .count() \
        .withColumnRenamed('count', f'count_by_{groupby_col[0]}') \
        .orderBy(F.col(*order_by_col).desc())
    return result_df

distribution_age = distribution(flat_df,['required_age'],['required_age'],['required_age'])

required_age,count_by_required_age
9,1
8,3
7,2
6,4
5,1
35,1
3,3
20,1
18,223
17,38


In [0]:
display(distribution_age.filter(F.col('required_age')<99).sort('required_age'))

required_age,count_by_required_age
0,55030
10,7
12,32
13,26
14,10
15,264
16,38
17,38
18,223
20,1


### Autres questions pertinentes

### Question 7 : Quels genres sont les plus représentés ?

In [0]:
genre_df = flat_df.withColumn('genres_exploded', F.explode(F.split(flat_df['genre'], ', ')))

genre_counts = genre_df.select('genres_exploded') \
    .groupBy('genres_exploded') \
    .count() \
    .withColumnRenamed('count', 'genre_count') \
    .orderBy(F.desc('genre_count'))

display(genre_counts.limit(5)) #top5

genres_exploded,genre_count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895


On s'aperçoit qu'il est difficile de conclure car il y a (logiquement) plus de jeux indépendants même si cette catégorie reste très vague.

### Quelle est la répartition des systèmes d'exploitation / plateformes ?

In [0]:
cols = ['windows','mac','linux']

os_df = flat_df.select(
    *(F.col(c).cast("integer").alias(c) for c in cols)
) \
.groupBy() \
.agg(
    F.sum(F.col('windows')).alias('total_windows'),
    F.sum(F.col('mac')).alias('total_mac'),
    F.sum(F.col('linux')).alias('total_linux')
)

# Showing the result
os_df.show()

+-------------+---------+-----------+
|total_windows|total_mac|total_linux|
+-------------+---------+-----------+
|        55676|    12770|       8458|
+-------------+---------+-----------+



Sans surprise Windows domine largement le marché des jeux vidéo, même si on remarque qu'un nombre important de jeux sont disponibles sous Mac ou Linux (notamment via l'influence de Proton et aujourd'hui du SteamDeck)